In [370]:
#!pip install nose
#for the below to work

    100% |████████████████████████████████| 163kB 3.5MB/s 


In [0]:
#from theano.sandbox import cuda
#cuda.use('gpu1')
# ensure that the colab run time ahs GPu in it

Omitted the above as they were a hinderance than anything else..  
Not sure if this lack of this configuration is what is causing theao defined funcitons to fail later on...  
https://groups.google.com/forum/#!topic/theano-users/yFk0-YUV44o

Cuda is a aprt of theano 0.9.0 and 1.0 is the latest..   
installing older version causes programs in theano to break down earlier

theano 1.0 doesnt allow the function to be created[not sure if it is a version issue of a global enviromment issue as shown in the link]

perhaps the whole thing needs to be set at the colab environment level itself.. ?

In [374]:
! pip install theano==0.9.0

  Found existing installation: Theano 1.0.0
    Uninstalling Theano-1.0.0:
      Successfully uninstalled Theano-1.0.0


In [109]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [110]:
! pip install bcolz

In [0]:
from keras.utils.data_utils import get_file

In [0]:
%matplotlib inline
#import utils; reload(utils)
#from utils import *
# do it the hard way install what you need get to know the code better
from __future__ import division, print_function

## Setup

We're going to download the collected works of Nietzsche to use as our data for this class.

In [113]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read()
print('corpus length:', len(text))

corpus length: 600901


In [114]:
chars = sorted(list(set(text))) # over kill .. use of set :D
vocab_size = len(chars)+1 # adding to accomodate teh nsertion of 0 later on
print('total chars:', vocab_size)

total chars: 86


Sometimes it's useful to have a zero value in the dataset, e.g. for padding

In [0]:
chars.insert(0, "\0")

In [116]:
len(chars)

86

In [117]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz'

In [118]:
''.join(chars[-6:-1]) # junck chars hence omitted.. not really readable

'\x86\xa4\xa6\xa9\xab'

Map from chars to indices and back again  

In [0]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

*idx* will be the data we use from now own - it simply converts all the characters to their index (based on the mapping above)  

Observe whats doine... create an array for every character c in text... get the character index from char_indices dic created in the list enumeration above

In [0]:
idx = [char_indices[c] for c in text]

In [121]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [122]:
''.join(indices_char[i] for i in idx[:70])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

## 3 char model

going on character basis.. as opposed to word basis  
The goal here.. given three chars find whats the fourth..

### Create inputs

Create a list of every 4th character, starting at the 0th, 1st, 2nd, then 3rd characters

In [0]:
cs=3 
# yeah.. instead of just a 3 character model.. change this to 5 ofr a 5 char model .. just make sure there are corespondingly 5+1 data arrays as seen below
c1_dat = [idx[i] for i in xrange(0, len(idx)-1-cs, cs)]
c2_dat = [idx[i+1] for i in xrange(0, len(idx)-1-cs, cs)]
c3_dat = [idx[i+2] for i in xrange(0, len(idx)-1-cs, cs)]
c4_dat = [idx[i+3] for i in xrange(0, len(idx)-1-cs, cs)]

Our inputs

In [0]:
import numpy as np

These are are inputs...

In [0]:
x1 = np.stack(c1_dat[:-2])
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])
# stack:: join a seq of arrays along a new axis.. as opposed to concatenate.. which does the same along the existing axis... refer manual

Our output

In [0]:
y = np.stack(c4_dat[:-2])

The first 4 inputs and outputs

In [127]:
x1[:4], x2[:4], x3[:4]

(array([40, 30, 29,  1]), array([42, 25,  1, 43]), array([29, 27,  1, 45]))

In [128]:
y[:4]

array([30, 29,  1, 40])

In [129]:
x1.shape, y.shape

((200297,), (200297,))

The number of latent factors to create (i.e. the size of the embedding matrix)

In [0]:
n_fac = 42

Create inputs and embedding outputs for each of our 3 character inputs

In [0]:
from keras.layers import Input, Embedding, merge
from keras.layers.core import Flatten, Dense
from keras.models import  Sequential, Model
from keras.optimizers import  Adam

In [0]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name) # input to the embedding layer
    emb = Embedding(n_in, n_out, input_length=1)(inp)# this and this alone takes care of the embedding generatation
    return inp, Flatten()(emb)

In [0]:
c1_in, c1 = embedding_input('c1', vocab_size, n_fac)
c2_in, c2 = embedding_input('c2', vocab_size, n_fac)
c3_in, c3 = embedding_input('c3', vocab_size, n_fac)

### Create and train model

Pick a size for our hidden state

In [0]:
n_hidden = 256

This is the 'green arrow' from the diagram - (refer the ppt in the video)the layer operation from input to hidden.

In [0]:
dense_in = Dense(n_hidden, activation='relu')

Our first hidden activation is simply this function applied to the result of the embedding of the first character.  
first character's array

In [0]:
c1_hidden = dense_in(c1)

This is the 'orange arrow' from our diagram -   
the layer operation from hidden to hidden.

In [0]:
dense_hidden = Dense(n_hidden, activation='tanh')

Our second and third hidden activations sum (merge by default does a sum) up the previous hidden state   
(after applying dense_hidden) to the new input state.



In [138]:
c2_dense = dense_in(c2) # C2 set/ array of all the 2nd characters ever./. in the input text
hidden_2 = dense_hidden(c1_hidden) #c1_hidden is the dense input c1 .. see above
c2_hidden = merge([c2_dense, hidden_2]) # keras layers merge

# difference b/w Merge and merge https://github.com/keras-team/keras/issues/3921

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  This is separate from the ipykernel package so we can avoid doing imports until


Repeat the above stunt :P agains for the 3rd character's arrays

In [139]:
c3_dense = dense_in(c3)
hidden_3 = dense_hidden(c2_hidden)
c3_hidden = merge([c3_dense, hidden_3])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  This is separate from the ipykernel package so we can avoid doing imports until


This is the 'blue arrow' from our diagram - the layer operation from hidden to output.  
The dense layer's output size is the entire vocab..

In [0]:
dense_out = Dense(vocab_size, activation='softmax')

The third hidden state is the input to our output layer.

In [0]:
c4_out = dense_out(c3_hidden)

In [0]:
model = Model([c1_in, c2_in, c3_in], c4_out)
# model defined the usual way..  input is an array.. of the input layer.. we get from the embeddings funciton above
# all of the above.. was to extract/derive the c4_out mentioned above..

In [0]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [0]:
model.optimizer.lr=0.000001

In [145]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/4
200297/200297 [==============================] - 22s 111us/step - loss: 4.4239
Epoch 2/4
188160/200297 [===========================>..] - ETA: 1s - loss: 4.3222

200297/200297 [==============================] - 22s 110us/step - loss: 4.3168
Epoch 3/4
200297/200297 [==============================] - 22s 112us/step - loss: 4.0777
Epoch 4/4
 56448/200297 [=======>......................] - ETA: 16s - loss: 3.8434

200297/200297 [==============================] - 22s 109us/step - loss: 3.6904


In [0]:
model.optimizer.lr=0.01

In [147]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)

Epoch 1/4
  1536/200297 [..............................] - ETA: 22s - loss: 3.4520

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


200297/200297 [==============================] - 22s 112us/step - loss: 3.3533
Epoch 2/4
200297/200297 [==============================] - 22s 108us/step - loss: 3.2046
Epoch 3/4
  8192/200297 [>.............................] - ETA: 21s - loss: 3.1646

200297/200297 [==============================] - 22s 110us/step - loss: 3.1449
Epoch 4/4
200297/200297 [==============================] - 23s 113us/step - loss: 3.1189


In [0]:
# model.optimizer.lr.set_value(0.000001)
model.optimizer.lr = 0.000001

In [149]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)

Epoch 1/4
  1472/200297 [..............................] - ETA: 23s - loss: 3.1216

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


200297/200297 [==============================] - 22s 110us/step - loss: 3.1035
Epoch 2/4
200297/200297 [==============================] - 23s 113us/step - loss: 3.0920
Epoch 3/4
  3392/200297 [..............................] - ETA: 23s - loss: 3.0719

200297/200297 [==============================] - 22s 111us/step - loss: 3.0825
Epoch 4/4
200297/200297 [==============================] - 22s 110us/step - loss: 3.0740


In [0]:
# model.optimizer.lr.set_value(0.01)
# Not sure why this doesnt work...^  
# But its a known issue and there is a variaiton in the set mentods vs assignment's resutls as seenin link below
# http://forums.fast.ai/t/different-training-accuracy-using-model-optimizer-lr-set-value-vs/240/6

model.optimizer.lr = 0.01

In [151]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)

Epoch 1/4
  1472/200297 [..............................] - ETA: 23s - loss: 3.0781

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


200297/200297 [==============================] - 22s 110us/step - loss: 3.0660
Epoch 2/4
200297/200297 [==============================] - 22s 109us/step - loss: 3.0583
Epoch 3/4
  8832/200297 [>.............................] - ETA: 20s - loss: 3.0494

200297/200297 [==============================] - 22s 111us/step - loss: 3.0507
Epoch 4/4
200297/200297 [==============================] - 22s 112us/step - loss: 3.0430


In [152]:
model.optimizer.lr = 0.001
model.fit([x1, x2, x3], y, batch_size=64, epochs=10)
# difficult to get the moss below the figure of 3 try what is used in the video... no empochs 10
# if ti works comment out the above.. cells

Epoch 1/10
200297/200297 [==============================] - 22s 110us/step - loss: 3.0351
Epoch 2/10
183104/200297 [==========================>...] - ETA: 1s - loss: 3.0283

200297/200297 [==============================] - 22s 111us/step - loss: 3.0270
Epoch 3/10
200297/200297 [==============================] - 22s 110us/step - loss: 3.0186
Epoch 4/10
 56768/200297 [=======>......................] - ETA: 16s - loss: 3.0198

200297/200297 [==============================] - 22s 111us/step - loss: 3.0099
Epoch 5/10
200297/200297 [==============================] - 22s 110us/step - loss: 3.0010
Epoch 6/10
 20864/200297 [==>...........................] - ETA: 19s - loss: 2.9997

200297/200297 [==============================] - 22s 109us/step - loss: 2.9918
Epoch 7/10
200297/200297 [==============================] - 22s 111us/step - loss: 2.9824
Epoch 8/10
 11136/200297 [>.............................] - ETA: 21s - loss: 2.9703

200297/200297 [==============================] - 22s 110us/step - loss: 2.9728
Epoch 9/10
200297/200297 [==============================] - 22s 109us/step - loss: 2.9630
Epoch 10/10
  9600/200297 [>.............................] - ETA: 21s - loss: 2.9715

200297/200297 [==============================] - 22s 110us/step - loss: 2.9531


### Test model

Throw in an input string.. and get the output charac..  
This model was based on characters.. hence

In [0]:
def get_next(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(arrs) # p is an array of 86 ouputs
    i = np.argmax(p) # take the max.. i.e most probable.. and op that coresponding thing form chars array
    print(i)
    return chars[i]

In [154]:
get_next('phi')

2


' '

In [155]:
#type(chars)
chars[57]

'd'

In [0]:
## nothing is coming... :/
# th:: e works. the rest seems to predict blank

In [157]:
get_next(' th')

2


' '

In [158]:
get_next(' an')

2


' '

In [159]:
get_next('sta')

2


' '

In [160]:
get_next('pho')

2


' '

## Our first RNN!
made from scratch.. keras

### Create inputs

This is the size of our unrolled RNN.  
Clustered together.. or rolled up .. together.. it would be an RNN

In [0]:
cs=8 # characters will be 8 and predict the 9th charac

For each of 0 through 7, create a list of every 8th character with that starting point.   
These will be the 8 inputs to out model.  
Just as commented above 

In [0]:
c_in_dat = [[idx[i+n] for i in xrange(0, len(idx)-1-cs, cs)] # range steps of cs, in idx which is all te text converted to numbers
            for n in range(cs)]

Then create a list of the next character in each of these series. This will be the labels for our model. output basically... if 1,2...7 then 8 is the op/label

In [0]:
c_out_dat = [idx[i+cs] for i in xrange(0, len(idx)-1-cs, cs)] # range.. steps of cs

In [0]:
xs = [np.stack(c[:-2]) for c in c_in_dat]

In [165]:
len(xs), xs[0].shape # all the x's inputs.. and the length of so..

(8, (75110,))

In [0]:
y = np.stack(c_out_dat[:-2])

So each column below is one series of 8 characters from the text.

In [167]:
[xs[n][:cs] for n in range(cs)]

[array([40,  1, 33,  2, 72, 67, 73,  2]),
 array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67])]

...and this is the next character after each sequence.

In [168]:
y[:cs]

array([ 1, 33,  2, 72, 67, 73,  2, 68])

In [0]:
n_fac = 42

### Create and train model
using the same techniques as above..

In [0]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name+'_in')
    emb = Embedding(n_in, n_out, input_length=1, name=name+'_emb')(inp)
    return inp, Flatten()(emb)

In [0]:
c_ins = [embedding_input('c'+str(n), vocab_size, n_fac) for n in range(cs)]

In [0]:
n_hidden = 256

In [173]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity') # initialised as idenity matrix.. not the default.. why.. becasue to observe equivalence in keras based model below
dense_out = Dense(vocab_size, activation='softmax')

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, kernel_initializer="identity", activation="relu")`
  


The first character of each sequence goes through dense_in(), to create our first hidden activations.

In [0]:
hidden = dense_in(c_ins[0][1])

Then for each successive layer we combine the output of dense_in()   
on the next character with the output of dense_hidden()   
on the current hidden state, to create the new hidden state. in a nice neat for loop

In [175]:
for i in range(1,cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  after removing the cwd from sys.path.


Putting the final hidden state through dense_out() gives us our output.

In [0]:
c_out = dense_out(hidden) #  like in the 3 character model.. the purpose of all of the above. was to get this c out layer..

So now we can create our model.

In [0]:
model = Model([c[0] for c in c_ins], c_out) # inputs in an array
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [178]:
model.fit(xs, y, batch_size=64, nb_epoch=12)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/12
75110/75110 [==============================] - 15s 202us/step - loss: 2.5362
Epoch 2/12
75110/75110 [==============================] - 15s 201us/step - loss: 2.2512
Epoch 3/12
33856/75110 [============>.................] - ETA: 8s - loss: 2.1549

75110/75110 [==============================] - 15s 199us/step - loss: 2.1470
Epoch 4/12
75110/75110 [==============================] - 15s 202us/step - loss: 2.0751
Epoch 5/12
48768/75110 [==================>...........] - ETA: 5s - loss: 2.0212

75110/75110 [==============================] - 16s 208us/step - loss: 2.0191
Epoch 6/12
75110/75110 [==============================] - 15s 201us/step - loss: 1.9729
Epoch 7/12
54016/75110 [====================>.........] - ETA: 4s - loss: 1.9275

75110/75110 [==============================] - 15s 204us/step - loss: 1.9324
Epoch 8/12
75110/75110 [==============================] - 15s 204us/step - loss: 1.8945
Epoch 9/12
53696/75110 [====================>.........] - ETA: 4s - loss: 1.8578

75110/75110 [==============================] - 15s 206us/step - loss: 1.8636
Epoch 10/12
75110/75110 [==============================] - 15s 198us/step - loss: 1.8345
Epoch 11/12
59200/75110 [======================>.......] - ETA: 3s - loss: 1.7995

75110/75110 [==============================] - 15s 197us/step - loss: 1.8097
Epoch 12/12
75110/75110 [==============================] - 15s 203us/step - loss: 1.7855


### Test model

This has a much lower error than the previous one.. that was built for predicting 4th char after 3 characters

In [0]:
def get_next(inp):
    idxs = [np.array(char_indices[c])[np.newaxis] for c in inp]
    p = model.predict(idxs)
    return chars[np.argmax(p)]

In [180]:
get_next('for thos')

'e'

This model works.. why does the one defined a few cells earlier work..?

In [181]:
get_next('part of ')

't'

In [182]:
get_next('queens a')

'n'

## Our first RNN with keras!
Keras As in  use the keras apis in particular.. SimpleRNN

In [0]:
n_hidden, n_fac, cs, vocab_size = (256, 42, 8, 86)

This is nearly exactly equivalent to the RNN we built ourselves in the previous section.

In [0]:
from keras.layers import SimpleRNN

In [185]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs), # get letter/character embeddings
        SimpleRNN(n_hidden, activation='relu', inner_init='identity'), # tadaaaaaann.....
        Dense(vocab_size, activation='softmax') # and output.. OMG soooo simpulll... provided you know what you are doing.. :P
    ])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `SimpleRNN` call to the Keras 2 API: `SimpleRNN(256, activation="relu", recurrent_initializer="identity")`
  This is separate from the ipykernel package so we can avoid doing imports until


In [186]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 8, 42)             3612      
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 256)               76544     
_________________________________________________________________
dense_15 (Dense)             (None, 86)                22102     
Total params: 102,258
Trainable params: 102,258
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [188]:
model.fit(np.concatenate([x[np.newaxis] for x in xs]).T, y, batch_size=64, nb_epoch=8)
# source http://forums.fast.ai/t/lesson-6-discussion/245/69
# original
#model.fit(np.concatenate(xs,axis=1), y, batch_size=64, nb_epoch=8)

Epoch 1/8
75110/75110 [==============================] - 17s 231us/step - loss: 2.8063
Epoch 2/8
75110/75110 [==============================] - 17s 225us/step - loss: 2.2801
Epoch 3/8
25408/75110 [=========>....................] - ETA: 11s - loss: 2.1351

75110/75110 [==============================] - 17s 229us/step - loss: 2.0784
Epoch 4/8
75110/75110 [==============================] - 17s 227us/step - loss: 1.9398
Epoch 5/8
39872/75110 [==============>...............] - ETA: 7s - loss: 1.8423

75110/75110 [==============================] - 17s 225us/step - loss: 1.8346
Epoch 6/8
75110/75110 [==============================] - 17s 227us/step - loss: 1.7547
Epoch 7/8
43264/75110 [================>.............] - ETA: 7s - loss: 1.6870

75110/75110 [==============================] - 17s 230us/step - loss: 1.6877
Epoch 8/8
75110/75110 [==============================] - 17s 224us/step - loss: 1.6335


In [0]:
def get_next_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = np.array(idxs)[np.newaxis,:]
    p = model.predict(arrs)[0]
    return chars[np.argmax(p)]

In [190]:
get_next_keras('this is ')

't'

In [191]:
get_next_keras('part of ')

't'

In [192]:
get_next_keras('ueens an')

'd'

In [193]:
get_next_keras('queens a')

'n'

The sentiment analysis use case:  
Given the above set up.. a sequence of characters.. and then predict the next..
when trained from a corpus...
similarly.. given a seq of words.. mapped on to a sentiment..   
use the RNN  trained on the above set.. to predict sentiments on samples of text not seen before.. 

## Returning sequences
explained as: 
return characters 2,3,4,5,6  
given characters 1,2,3,4,5    
as input



### Create inputs

To use a sequence model, we can leave our input unchanged - but we have to change our output to a sequence (of course!)

Here, c_out_dat is identical to c_in_dat, but moved across 1 character.

In [0]:
#c_in_dat = [[idx[i+n] for i in xrange(0, len(idx)-1-cs, cs)]
#            for n in range(cs)]
c_out_dat = [[idx[i+n] for i in xrange(1, len(idx)-cs, cs)]# shifted over by 1
            for n in range(cs)]

In [0]:
ys = [np.stack(c[:-2]) for c in c_out_dat]

Reading down each column shows one set of inputs and outputs.

In [196]:
[xs[n][:cs] for n in range(cs)]

[array([40,  1, 33,  2, 72, 67, 73,  2]),
 array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67])]

In [197]:
[ys[n][:cs] for n in range(cs)]

[array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67]),
 array([ 1, 33,  2, 72, 67, 73,  2, 68])]

### Create and train model

In [198]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax', name='output')

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, kernel_initializer="identity", activation="relu")`
  


We're going to pass a vector of all zeros as our starting point - here's our input layers for that:

In [0]:
inp1 = Input(shape=(n_fac,), name='zeros')
hidden = dense_in(inp1)

# taking an n character input: rather than n-1: hence set the initial input as a vector as zeroes
# initialize as 0s

In [200]:
outs = []

for i in range(cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden], mode='sum')
    # every layer now has an output
    outs.append(dense_out(hidden))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:6: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  


In [0]:
model = Model([inp1] + [c[0] for c in c_ins], outs)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [202]:
zeros = np.tile(np.zeros(n_fac), (len(xs[0]),1))
zeros.shape

(75110, 42)

In [203]:
model.fit([zeros]+xs, ys, batch_size=64, nb_epoch=12)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/12
75110/75110 [==============================] - 27s 358us/step - loss: 20.0608 - output_loss: 2.4405
Epoch 2/12
44416/75110 [================>.............] - ETA: 10s - loss: 18.0105 - output_loss: 2.1468

75110/75110 [==============================] - 27s 356us/step - loss: 17.8570 - output_loss: 2.1290
Epoch 3/12
71232/75110 [===========================>..] - ETA: 1s - loss: 17.2630 - output_loss: 2.0244

75110/75110 [==============================] - 27s 355us/step - loss: 17.2480 - output_loss: 2.0229
Epoch 4/12
75110/75110 [==============================] - 27s 357us/step - loss: 16.8725 - output_loss: 1.9521
Epoch 5/12
 3968/75110 [>.............................] - ETA: 25s - loss: 16.6668 - output_loss: 1.8829

75110/75110 [==============================] - 27s 354us/step - loss: 16.6119 - output_loss: 1.9020
Epoch 6/12
50688/75110 [===================>..........] - ETA: 8s - loss: 16.4400 - output_loss: 1.8651

75110/75110 [==============================] - 27s 356us/step - loss: 16.4256 - output_loss: 1.8644
Epoch 7/12
72000/75110 [===========================>..] - ETA: 1s - loss: 16.2813 - output_loss: 1.8371

75110/75110 [==============================] - 27s 355us/step - loss: 16.2829 - output_loss: 1.8373
Epoch 8/12
75110/75110 [==============================] - 27s 357us/step - loss: 16.1713 - output_loss: 1.8119
Epoch 9/12
 5504/75110 [=>............................] - ETA: 25s - loss: 15.9992 - output_loss: 1.7973

75110/75110 [==============================] - 27s 355us/step - loss: 16.0774 - output_loss: 1.7936
Epoch 10/12
51840/75110 [===================>..........] - ETA: 8s - loss: 15.9835 - output_loss: 1.7732

75110/75110 [==============================] - 27s 358us/step - loss: 15.9992 - output_loss: 1.7776
Epoch 11/12
72320/75110 [===========================>..] - ETA: 0s - loss: 15.9328 - output_loss: 1.7619

75110/75110 [==============================] - 27s 358us/step - loss: 15.9338 - output_loss: 1.7633
Epoch 12/12
75110/75110 [==============================] - 27s 356us/step - loss: 15.8730 - output_loss: 1.7501


Expected multiple losses.. 1 for each of the 8 char layers

### Test model

In [0]:
def get_nexts(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict([np.zeros(n_fac)[np.newaxis,:]] + arrs)
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [208]:
get_nexts(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 't', ' ', 'i', 'n', ' ']

In [209]:
get_nexts(' part of')

[' ', 'p', 'a', 'r', 't', ' ', 'o', 'f']


['t', 'o', 'r', 't', 'i', 'o', 'f', ' ']

In [211]:
get_nexts('d mornin')

['d', ' ', 'm', 'o', 'r', 'n', 'i', 'n']


[' ', 't', 'a', 'r', 'e', ' ', 's', 'g']

Very off when looked at closely :)

### Sequence model with keras
make the same as above with keras this time tho..

In [226]:
n_hidden, n_fac, cs, vocab_size

(256, 42, 8, 86)

To convert our previous keras model into a sequence model,   
simply add the 'return_sequences=True' parameter, and add TimeDistributed() around our dense layer.

In [0]:
from keras.layers import TimeDistributed

In [213]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs),
        SimpleRNN(n_hidden, return_sequences=True, activation='relu', inner_init='identity'),# return sequences set to true
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `SimpleRNN` call to the Keras 2 API: `SimpleRNN(256, return_sequences=True, activation="relu", recurrent_initializer="identity")`
  This is separate from the ipykernel package so we can avoid doing imports until


In [214]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 8, 42)             3612      
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 8, 256)            76544     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 8, 86)             22102     
Total params: 102,258
Trainable params: 102,258
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [216]:
xs[0].shape

(75110,)

In [0]:
x_rnn=np.stack(np.squeeze(xs), axis=1) # dimensionality changes to fit into the rnn inputs.. gotta chk these out more..
y_rnn=np.atleast_3d(np.stack(ys, axis=1))

In [222]:
x_rnn.shape, y_rnn.shape

((75110, 8), (75110, 8, 1))

In [223]:
model.fit(x_rnn, y_rnn, batch_size=64, nb_epoch=8)

Epoch 1/8
75110/75110 [==============================] - 31s 411us/step - loss: 2.4303
Epoch 2/8
42880/75110 [================>.............] - ETA: 13s - loss: 2.0304

75110/75110 [==============================] - 31s 410us/step - loss: 1.9982
Epoch 3/8
69888/75110 [==========================>...] - ETA: 2s - loss: 1.8845

75110/75110 [==============================] - 31s 410us/step - loss: 1.8830
Epoch 4/8
75110/75110 [==============================] - 31s 411us/step - loss: 1.8221
Epoch 5/8
 3520/75110 [>.............................] - ETA: 29s - loss: 1.7960

75110/75110 [==============================] - 31s 412us/step - loss: 1.7827
Epoch 6/8
50688/75110 [===================>..........] - ETA: 10s - loss: 1.7579

75110/75110 [==============================] - 31s 414us/step - loss: 1.7560
Epoch 7/8
73280/75110 [============================>.] - ETA: 0s - loss: 1.7362

75110/75110 [==============================] - 31s 409us/step - loss: 1.7362
Epoch 8/8
75110/75110 [==============================] - 31s 414us/step - loss: 1.7203


In [0]:
def get_nexts_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arr = np.array(idxs)[np.newaxis,:]
    p = model.predict(arr)[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [225]:
get_nexts_keras(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 'n', ' ', 'i', 'n', ' ']

### One-hot sequence model with keras

This is the keras version of the theano model that we're about to create. : why theano..?
   
   theano doenst require a rolled out version of the model.. as we'll see later on

In [227]:
model=Sequential([
        SimpleRNN(n_hidden, return_sequences=True, input_shape=(cs, vocab_size),
                  activation='relu', inner_init='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
model.compile(loss='categorical_crossentropy', optimizer=Adam()) # observe the different type of loss used

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `SimpleRNN` call to the Keras 2 API: `SimpleRNN(256, return_sequences=True, activation="relu", input_shape=(8, 86), recurrent_initializer="identity")`
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
from keras.utils.np_utils import to_categorical

In [229]:
oh_ys = [to_categorical(o, vocab_size) for o in ys] # no embedding layer.. hence one hotting the inputs and outputs.. :)
oh_y_rnn=np.stack(oh_ys, axis=1)

oh_xs = [to_categorical(o, vocab_size) for o in xs]
oh_x_rnn=np.stack(oh_xs, axis=1)

oh_x_rnn.shape, oh_y_rnn.shape

((75110, 8, 86), (75110, 8, 86))

In [230]:
model.fit(oh_x_rnn, oh_y_rnn, batch_size=64, nb_epoch=8)

Epoch 1/8
75110/75110 [==============================] - 41s 551us/step - loss: 2.4487
Epoch 2/8
38208/75110 [==============>...............] - ETA: 19s - loss: 2.0754

75110/75110 [==============================] - 41s 541us/step - loss: 2.0404
Epoch 3/8


75110/75110 [==============================] - 40s 538us/step - loss: 1.9273
Epoch 4/8
73664/75110 [============================>.] - ETA: 0s - loss: 1.8620

75110/75110 [==============================] - 41s 540us/step - loss: 1.8619
Epoch 5/8
75110/75110 [==============================] - 41s 543us/step - loss: 1.8191
Epoch 6/8
 1856/75110 [..............................] - ETA: 39s - loss: 1.7942

75110/75110 [==============================] - 41s 542us/step - loss: 1.7873
Epoch 7/8
47296/75110 [=================>............] - ETA: 15s - loss: 1.7638

75110/75110 [==============================] - 41s 541us/step - loss: 1.7637
Epoch 8/8
68032/75110 [==========================>...] - ETA: 3s - loss: 1.7447

75110/75110 [==============================] - 41s 544us/step - loss: 1.7449


In [0]:
def get_nexts_oh(inp):
    idxs = np.array([char_indices[c] for c in inp])
    arr = to_categorical(idxs, vocab_size)
    p = model.predict(arr[np.newaxis,:])[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [232]:
get_nexts_oh(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 'n', ' ', 'c', 'n', ' ']

we get the same  answer as the one with the embedding layer..  
and the non on ehot encoded version of the model above

## Stateful model with keras

In [0]:
bs=64

A stateful model is easy to create (just add "stateful=True") but harder to train. We had to add batchnorm and use LSTM to get reasonable results.  
When using stateful in keras, you have to also add 'batch_input_shape' to the first layer, and fix the batch size there.  
Stateful: do not reset the hidding  layers' activations to 0 after a pass throug.. thus getting/capturing long term dependencies..

In [0]:
from keras.layers.normalization import BatchNormalization
from keras.layers import  LSTM, Bidirectional

In [0]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs, batch_input_shape=(bs,8)),
        BatchNormalization(),
        LSTM(n_hidden, return_sequences=True, stateful=True),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])

In [0]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

Since we're using a fixed batch shape, we have to ensure our inputs and outputs are a even multiple of the batch size.

In [0]:
mx = len(x_rnn)//bs*bs

We also set shiffle to false becasue : default is true..  
and at that default state. the sequences of 8 characters (or whatever other number) are being passed at random..  
This setting along with the stateful setting.. as mentioned above.. would leave the weights unchanged after each pass of 8 characters  
Takes a lot of time.. since its not randomised.. and the stateful parameter is siet to true..

In [239]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False)

Epoch 1/4
75072/75072 [==============================] - 68s 901us/step - loss: 2.2904
Epoch 2/4
 2560/75072 [>.............................] - ETA: 1:05 - loss: 2.1701

75072/75072 [==============================] - 67s 893us/step - loss: 2.0338
Epoch 3/4
 8320/75072 [==>...........................] - ETA: 59s - loss: 2.0190

75072/75072 [==============================] - 67s 888us/step - loss: 1.9592
Epoch 4/4
10944/75072 [===>..........................] - ETA: 56s - loss: 1.9550

75072/75072 [==============================] - 67s 887us/step - loss: 1.9171


In [0]:
model.optimizer.lr=1e-4

In [241]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False)

Epoch 1/4
75072/75072 [==============================] - 66s 884us/step - loss: 1.8886
Epoch 2/4
 2304/75072 [..............................] - ETA: 1:04 - loss: 1.9472

75072/75072 [==============================] - 67s 886us/step - loss: 1.8680
Epoch 3/4
 8192/75072 [==>...........................] - ETA: 59s - loss: 1.8917

75072/75072 [==============================] - 67s 887us/step - loss: 1.8518
Epoch 4/4
10880/75072 [===>..........................] - ETA: 57s - loss: 1.8650

75072/75072 [==============================] - 66s 885us/step - loss: 1.8382


In [242]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False)

Epoch 1/4
75072/75072 [==============================] - 66s 884us/step - loss: 1.8268
Epoch 2/4
 2304/75072 [..............................] - ETA: 1:05 - loss: 1.8861

75072/75072 [==============================] - 66s 884us/step - loss: 1.8170
Epoch 3/4
 8192/75072 [==>...........................] - ETA: 58s - loss: 1.8439

75072/75072 [==============================] - 66s 884us/step - loss: 1.8081
Epoch 4/4
10880/75072 [===>..........................] - ETA: 57s - loss: 1.8239

75072/75072 [==============================] - 67s 894us/step - loss: 1.8000


In [0]:
# how to test the above..?

Check out the notebook Char RNN

In [0]:
# theano is the back end.. we've reached apoint where.. you keep adding more and more stuff/layers on top of keras.. and it becomes easier to use theano itself.. :/

## Theano RNN

In [0]:
n_input = vocab_size
n_output = vocab_size

Using raw theano, we have to create our weight matrices and bias vectors ourselves - here are the functions we'll use to do so (using glorot initialization).

The return values are wrapped in `shared()`, which is how we tell theano that it can manage this data (copying it to and from the GPU as necessary).

In [0]:
def init_wgts(rows, cols): 
    scale = math.sqrt(2/rows)
    return shared(normal(scale=scale, size=(rows, cols)).astype(np.float32))
def init_bias(rows): 
    return shared(np.zeros(rows, dtype=np.float32))

We return the weights and biases together as a tuple. For the hidden weights, we'll use an identity initialization (as recommended by [Hinton](https://arxiv.org/abs/1504.00941).)

In [0]:
def wgts_and_bias(n_in, n_out): 
    return init_wgts(n_in, n_out), init_bias(n_out)
def id_and_bias(n): 
    return shared(np.eye(n, dtype=np.float32)), init_bias(n)

Theano doesn't actually do any computations until we explicitly compile and evaluate the function (at which point it'll be turned into CUDA code and sent off to the GPU). So our job is to describe the computations that we'll want theano to do - the first step is to tell theano what inputs we'll be providing to our computation:

In [0]:
t_inp = T.matrix('inp')
t_outp = T.matrix('outp')
t_h0 = T.vector('h0')
lr = T.scalar('lr')

all_args = [t_h0, t_inp, t_outp, lr]

Now we're ready to create our intial weight matrices.

In [0]:
W_h = id_and_bias(n_hidden)
W_x = wgts_and_bias(n_input, n_hidden)
W_y = wgts_and_bias(n_hidden, n_output)
w_all = list(chain.from_iterable([W_h, W_x, W_y]))

Theano handles looping by using the [GPU scan](http://http.developer.nvidia.com/GPUGems3/gpugems3_ch39.html) operation. We have to tell theano what to do at each step through the scan - this is the function we'll use, which does a single forward pass for one character:

In [0]:
def step(x, h, W_h, b_h, W_x, b_x, W_y, b_y):
    # Calculate the hidden activations
    h = nnet.relu(T.dot(x, W_x) + b_x + T.dot(h, W_h) + b_h)
    # Calculate the output activations
    y = nnet.softmax(T.dot(h, W_y) + b_y)
    # Return both (the 'Flatten()' is to work around a theano bug)
    return h, T.flatten(y, 1)

Now we can provide everything necessary for the scan operation, so we can setup that up - we have to pass in the function to call at each step, the sequence to step through, the initial values of the outputs, and any other arguments to pass to the step function.

In [298]:
[v_h, v_y], _ = theano.scan(step, sequences=t_inp, 
                            outputs_info=[t_h0, None], non_sequences=w_all)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: UserWarning: DEPRECATION: If x is a vector, Softmax will not automatically pad x anymore in next releases. If you need it, please do it manually. The vector case is gonna be supported soon and the output will be a vector.
  """


We can now calculate our loss function, and *all* of our gradients, with just a couple of lines of code!

In [0]:
error = nnet.categorical_crossentropy(v_y, t_outp).sum()
g_all = T.grad(error, w_all)

We even have to show theano how to do SGD - so we set up this dictionary of updates to complete after every forward pass, which apply to standard SGD update rule to every weight.

In [0]:
def upd_dict(wgts, grads, lr): 
    return OrderedDict({w: w-g*lr for (w,g) in zip(wgts,grads)})

upd = upd_dict(w_all, g_all, lr)

We're finally ready to compile the function!

In [302]:
#fn = theano.function(all_args, error, updates=upd, allow_input_downcast=True)

TypeError: ignored

Function creation doesnt wwork..  
Tried downgrading to 0.9.0 no effect.. data type mismatch

In [303]:
X = oh_x_rnn
Y = oh_y_rnn
X.shape, Y.shape

((75110, 8, 86), (75110, 8, 86))

To use it, we simply loop through our input data, calling the function compiled above, and printing our progress from time to time.

In [304]:
err=0.0; l_rate=0.01
for i in range(len(X)): 
    err+=fn(np.zeros(n_hidden), X[i], Y[i], l_rate)
    if i % 1000 == 999: 
        print ("Error:{:.3f}".format(err/1000))
        err=0.0

NameError: ignored

In [0]:
f_y = theano.function([t_h0, t_inp], v_y, allow_input_downcast=True)

Funnyli enough the output fn creation worked.. :D  


In [0]:
pred = np.argmax(f_y(np.zeros(n_hidden), X[6]), axis=1)

In [0]:
act = np.argmax(X[6], axis=1)

In [308]:
[indices_char[o] for o in act]

['t', 'h', 'e', 'n', '?', ' ', 'I', 's']

In [309]:
[indices_char[o] for o in pred]

['F', '0', '0', 'c', '4', 'c', 'c', 'L']

Since the input wasn't used in training the mode... the output is gibberish :/

## Pure python RNN!

### Set up basic functions

Now we're going to try to repeat the above theano RNN, using just pure python (and numpy).   
we have to do everything ourselves, including defining the basic functions of a neural net!  
Below are all of the definitions, along with tests to check that they give the same answers as theano. The functions ending in `_d` are the derivatives of each function.

In [0]:
def sigmoid(x): return 1/(1+np.exp(-x))
def sigmoid_d(x): 
    output = sigmoid(x)
    return output*(1-output)

define sigmoid fn

In [0]:
def relu(x): return np.maximum(0., x)
def relu_d(x): return (x > 0.)*1.

refine relu 

In [312]:
relu(np.array([3.,-3.])), relu_d(np.array([3.,-3.]))

(array([3., 0.]), array([1., 0.]))

In [0]:
def dist(a,b): return pow(a-b,2)
def dist_d(a,b): return 2*(a-b)

In [0]:
import pdb # python debugger.. cool tool.. much like the eclipse de bugger.. check values of what ever on the notebook

In [0]:
eps = 1e-7
def x_entropy(pred, actual): 
    return -np.sum(actual * np.log(np.clip(pred, eps, 1-eps)))
def x_entropy_d(pred, actual): return -actual/pred

In [0]:
def softmax(x): return np.exp(x)/np.exp(x).sum()

In [0]:
def softmax_d(x):
    sm = softmax(x)
    res = np.expand_dims(-sm,-1)*sm
    res[np.diag_indices_from(res)] = sm*(1-sm)
    return res

In [318]:
test_preds = np.array([0.2,0.7,0.1])
test_actuals = np.array([0.,1.,0.])
nnet.categorical_crossentropy(test_preds, test_actuals).eval()

array(0.35667494)

In [319]:
x_entropy(test_preds, test_actuals)

0.35667494393873245

In [0]:
test_inp = T.dvector()
test_out = nnet.categorical_crossentropy(test_inp, test_actuals)
test_grad = theano.function([test_inp], T.grad(test_out, test_inp))

In [321]:
test_grad(test_preds)

array([-0.        , -1.42857143, -0.        ])

In [322]:
x_entropy_d(test_preds, test_actuals)

array([-0.        , -1.42857143, -0.        ])

In [0]:
from numpy.random import random

In [0]:
pre_pred = random(oh_x_rnn[0][0].shape)
preds = softmax(pre_pred)
actual = oh_x_rnn[0][0]

In [326]:
np.allclose(softmax_d(pre_pred).dot(x_entropy_d(preds,actual)), preds-actual)

True

In [327]:
softmax(test_preds)

array([0.28140804, 0.46396343, 0.25462853])

In [328]:
nnet.softmax(test_preds).eval()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning: DEPRECATION: If x is a vector, Softmax will not automatically pad x anymore in next releases. If you need it, please do it manually. The vector case is gonna be supported soon and the output will be a vector.
  """Entry point for launching an IPython kernel.


array([[0.28140804, 0.46396343, 0.25462853]])

In [329]:
test_out = T.flatten(nnet.softmax(test_inp))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning: DEPRECATION: If x is a vector, Softmax will not automatically pad x anymore in next releases. If you need it, please do it manually. The vector case is gonna be supported soon and the output will be a vector.
  """Entry point for launching an IPython kernel.


In [0]:
test_grad = theano.function([test_inp], theano.gradient.jacobian(test_out, test_inp))

In [331]:
test_grad(test_preds)

array([[ 0.20221756, -0.13056304, -0.07165452],
       [-0.13056304,  0.24870137, -0.11813832],
       [-0.07165452, -0.11813832,  0.18979284]])

In [332]:
softmax_d(test_preds)

array([[ 0.20221756, -0.13056304, -0.07165452],
       [-0.13056304,  0.24870137, -0.11813832],
       [-0.07165452, -0.11813832,  0.18979284]])

In [0]:
act=relu
act_d = relu_d # activation fn is relu, and the activation function's derivative is .. assigned as relu_d

In [0]:
loss=x_entropy
loss_d=x_entropy_d

We also have to define our own scan function. Since we're not worrying about running things in parallel (as in theano), it's very simple to implement:

In [0]:
def scan(fn, start, seq):
    res = []
    prev = start
    for s in seq:
        app = fn(prev, s)
        res.append(app)
        prev = app
    return res

...for instance, `scan` on `+` is the cumulative sum.

In [336]:
scan(lambda prev,curr: prev+curr, 0, range(5))

[0, 1, 3, 6, 10]

### Set up training

Let's now build the functions to do the forward and backward passes of our RNN. First, define our data and shape.

In [0]:
inp = oh_x_rnn
outp = oh_y_rnn
n_input = vocab_size
n_output = vocab_size

In [338]:
inp.shape, outp.shape

((75110, 8, 86), (75110, 8, 86))

Here's the function to do a single forward pass of an RNN, for a single character.

In [0]:
def one_char(prev, item):
    # Previous state
    tot_loss, pre_hidden, pre_pred, hidden, ypred = prev
    # Current inputs and output
    x, y = item
    pre_hidden = np.dot(x,w_x) + np.dot(hidden,w_h)
    hidden = act(pre_hidden)
    pre_pred = np.dot(hidden,w_y)
    ypred = softmax(pre_pred)
    return (
        # Keep track of loss so we can report it
        tot_loss+loss(ypred, y),
        # Used in backprop
        pre_hidden, pre_pred, 
        # Used in next iteration
        hidden, 
        # To provide predictions
        ypred)

We use `scan` to apply the above to a whole sequence of characters.

In [0]:
def get_chars(n): return zip(inp[n], outp[n])
def one_fwd(n): return scan(one_char, (0,0,0,np.zeros(n_hidden),0), get_chars(n))# one forward pass
# this is stateless, stateful.. add the retun from the scan funciton insteadof the zeros
# not a good idea tho.. gradient explostion etc

Now we can define the backward step.  
We use a loop to go through every element of the sequence.  
The derivatives are applying the chain rule to each step, and accumulating the gradients across the sequence.

In [0]:
# "Columnify" a vector
def col(x): return x[:,newaxis]

def one_bkwd(args, n):
    global w_x,w_y,w_h

    i=inp[n]  # 8x86
    o=outp[n] # 8x86
    d_pre_hidden = np.zeros(n_hidden) # 256
    for p in reversed(range(len(i))):
        totloss, pre_hidden, pre_pred, hidden, ypred = args[p]
        x=i[p] # 86
        y=o[p] # 86
        d_pre_pred = softmax_d(pre_pred).dot(loss_d(ypred,y))  # 86
        d_pre_hidden = (np.dot(d_pre_hidden, w_h.T) ## matrix multiplicaiton derivative is matrix mult with a transpose..
                        + np.dot(d_pre_pred,w_y.T)) * act_d(pre_hidden) # 256 # this : deivative with respect to the outputs.. above. deivative w.r.t hidden weights

        # d(loss)/d(w_y) = d(loss)/d(pre_pred) * d(pre_pred)/d(w_y)
        w_y -= col(hidden) * d_pre_pred * alpha
        # d(loss)/d(w_h) = d(loss)/d(pre_hidden[p-1]) * d(pre_hidden[p-1])/d(w_h)
        if (p>0): w_h -= args[p-1][3].dot(d_pre_hidden) * alpha
        w_x -= col(x)*d_pre_hidden * alpha
    return d_pre_hidden

Now we can set up our initial weight matrices. Note that we're not using bias at all in this example, in order to keep things simpler.

In [0]:
scale=math.sqrt(2./n_input)
w_x = normal(scale=scale, size=(n_input,n_hidden))
w_y = normal(scale=scale, size=(n_hidden, n_output))
w_h = np.eye(n_hidden, dtype=np.float32)

Our loop looks much like the theano loop in the previous section, except that we have to call the backwards step ourselves.

In [0]:
from numpy import newaxis

In [345]:
overallError=0
alpha=0.0001
for n in range(10000):
    res = one_fwd(n)
    overallError+=res[-1][0]
    deriv = one_bkwd(res, n)
    if(n % 1000 == 999):
        print ("Error:{:.4f}; Gradient:{:.5f}".format(
                overallError/1000, np.linalg.norm(deriv)))
        overallError=0

Error:35.7141; Gradient:2.11107
Error:35.4483; Gradient:2.82171
Error:34.2573; Gradient:4.22795
Error:32.3743; Gradient:3.46695
Error:30.9945; Gradient:3.94601
Error:30.2121; Gradient:3.40161
Error:29.2409; Gradient:3.99968
Error:28.6919; Gradient:3.36748
Error:28.1954; Gradient:3.41394
Error:28.2226; Gradient:3.70966


## Keras GRU

Identical to the last keras rnn, but a GRU!

checke out https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

In [0]:
from keras.layers import  GRU

In [0]:
model=Sequential([
        GRU(n_hidden, return_sequences=True, input_shape=(cs, vocab_size),
                  activation='relu' ),
                  #activation='relu', inner_init='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
model.compile(loss='categorical_crossentropy', optimizer=Adam())

Error idenitity .. matris can be used with 2d matrices only.. hence blanked out..

In [350]:
model.fit(oh_x_rnn, oh_y_rnn, batch_size=64, nb_epoch=8)

Epoch 1/8
75110/75110 [==============================] - 72s 962us/step - loss: 2.4018
Epoch 2/8
 2368/75110 [..............................] - ETA: 1:08 - loss: 2.0647

75110/75110 [==============================] - 71s 942us/step - loss: 1.9860
Epoch 3/8
 8192/75110 [==>...........................] - ETA: 1:02 - loss: 1.9123

75110/75110 [==============================] - 71s 940us/step - loss: 1.8755
Epoch 4/8
10816/75110 [===>..........................] - ETA: 1:00 - loss: 1.8306

75110/75110 [==============================] - 71s 939us/step - loss: 1.8127
Epoch 5/8
11968/75110 [===>..........................] - ETA: 59s - loss: 1.7774

75110/75110 [==============================] - 71s 941us/step - loss: 1.7709
Epoch 6/8
12480/75110 [===>..........................] - ETA: 59s - loss: 1.7450

75110/75110 [==============================] - 71s 939us/step - loss: 1.7405
Epoch 7/8
12736/75110 [====>.........................] - ETA: 58s - loss: 1.7185

75110/75110 [==============================] - 71s 943us/step - loss: 1.7160
Epoch 8/8
12864/75110 [====>.........................] - ETA: 58s - loss: 1.6916

75110/75110 [==============================] - 71s 943us/step - loss: 1.6970


In [351]:
get_nexts_oh(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 's', ' ', 'c', 's', ' ']

## Theano GRU

GRU: gated recurrent unit...

### Separate weights

The theano GRU looks just like the simple theano RNN, except for the use of the reset and update gates. Each of these gates requires its own hidden and input weights, so we add those to our weight matrices.

In [0]:
W_h = id_and_bias(n_hidden)
W_x = init_wgts(n_input, n_hidden)
W_y = wgts_and_bias(n_hidden, n_output)
rW_h = init_wgts(n_hidden, n_hidden)
rW_x = wgts_and_bias(n_input, n_hidden)
uW_h = init_wgts(n_hidden, n_hidden)
uW_x = wgts_and_bias(n_input, n_hidden)
w_all = list(chain.from_iterable([W_h, W_y, uW_x, rW_x]))
w_all.extend([W_x, uW_h, rW_h])

In [366]:
type(uW_x.dtype)

AttributeError: ignored

In [365]:
type(uW_h.dtype)

str

Here's the definition of a gate - it's just a sigmoid applied to the addition of the dot products of the input vectors.

In [0]:
def gate(x, h, W_h, W_x, b_x):
    return nnet.sigmoid(T.dot(x, W_x) + b_x + T.dot(h, W_h))

Our step is nearly identical to before, except that we multiply our hidden state by our reset gate, and we update our hidden state based on the update gate.

In [0]:
def step(x, h, W_h, b_h, W_y, b_y, uW_x, ub_x, rW_x, rb_x, W_x, uW_h, rW_h):
    reset = gate(x, h, rW_h, rW_x, rb_x)
    update = gate(x, h, uW_h, uW_x, ub_x)
    h_new = gate(x, h * reset, W_h, W_x, b_h)
    h = update*h + (1-update)*h_new
    y = nnet.softmax(T.dot(h, W_y) + b_y)
    return h, T.flatten(y, 1)

Everything from here on is identical to our simple RNN in theano.

In [355]:
[v_h, v_y], _ = theano.scan(step, sequences=t_inp, 
                            outputs_info=[t_h0, None], non_sequences=w_all)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:6: UserWarning: DEPRECATION: If x is a vector, Softmax will not automatically pad x anymore in next releases. If you need it, please do it manually. The vector case is gonna be supported soon and the output will be a vector.
  


In [0]:
error = nnet.categorical_crossentropy(v_y, t_outp).sum()
g_all = T.grad(error, w_all)

In [0]:
!THEANO_FLAGS='floatX=float32'

In [368]:
upd = upd_dict(w_all, g_all, lr)
fn = theano.function(all_args, error, updates=upd, allow_input_downcast=True)

TypeError: ignored

Same funny error message as above.. when defining the fn:

('An update must have the same type as the original shared variable (shared_var=<TensorType(float32, vector)>, shared_var.type=TensorType(float32, vector), update_val=Elemwise{sub,no_inplace}.0, update_val.type=TensorType(float64, vector)).', 'If the difference is related to the broadcast pattern, you can call the tensor.unbroadcast(var, axis_to_unbroadcast[, ...]) function to remove broadcastable dimensions.')

In [358]:
err=0.0; l_rate=0.1
for i in range(len(X)): 
    err+=fn(np.zeros(n_hidden), X[i], Y[i], l_rate)
    if i % 1000 == 999: 
        l_rate *= 0.95
        print ("Error:{:.2f}".format(err/1000))
        err=0.0

NameError: ignored

### Combined weights

We can make the previous section simpler and faster by concatenating the hidden and input matrices and inputs together. We're not going to step through this cell by cell - you'll see it's identical to the previous section except for this concatenation.

In [0]:
W = (shared(np.concatenate([np.eye(n_hidden), normal(size=(n_input, n_hidden))])
            .astype(np.float32)), init_bias(n_hidden))

rW = wgts_and_bias(n_input+n_hidden, n_hidden)
uW = wgts_and_bias(n_input+n_hidden, n_hidden)
W_y = wgts_and_bias(n_hidden, n_output)
w_all = list(chain.from_iterable([W, W_y, uW, rW]))

In [0]:
def gate(m, W, b): return nnet.sigmoid(T.dot(m, W) + b)

In [0]:
def step(x, h, W, b, W_y, b_y, uW, ub, rW, rb):
    m = T.concatenate([h, x])
    reset = gate(m, rW, rb)
    update = gate(m, uW, ub)
    m = T.concatenate([h*reset, x])
    h_new = gate(m, W, b)
    h = update*h + (1-update)*h_new
    y = nnet.softmax(T.dot(h, W_y) + b_y)
    return h, T.flatten(y, 1)

In [0]:
[v_h, v_y], _ = theano.scan(step, sequences=t_inp, 
                            outputs_info=[t_h0, None], non_sequences=w_all)

In [0]:
def upd_dict(wgts, grads, lr): 
    return OrderedDict({w: w-g*lr for (w,g) in zip(wgts,grads)})

In [0]:
error = nnet.categorical_crossentropy(v_y, t_outp).sum()
g_all = T.grad(error, w_all)

In [0]:
upd = upd_dict(w_all, g_all, lr)
fn = theano.function(all_args, error, updates=upd, allow_input_downcast=True)

In [0]:
err=0.0; l_rate=0.01
for i in range(len(X)): 
    err+=fn(np.zeros(n_hidden), X[i], Y[i], l_rate)
    if i % 1000 == 999: 
        print ("Error:{:.2f}".format(err/1000))
        err=0.0

### End